### Install python package dependencies

In [1]:
!pip install kaleido plotly==5.11.0

### Import packages to graph results

In [2]:
from glob import glob
import pandas as pd
import json
#import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = 'iframe'
# import plotly.offline as py
# py.init_notebook_mode(connected=True)
# Used for debug
# from pprint import pprint

### Set default color palete and size for making plots

In [3]:
colorPallet = ["rgba(151, 209, 233, 255)", "rgba(0, 120, 179, 255)", "rgba(179, 223, 146, 255)", "rgba(49, 169, 90, 255)", "rgba(227, 136, 220, 255)", "rgba(127, 0, 255, 255)", "rgba(255, 128, 0, 255)"]
bgColor = 'rgba(245,245,255,255)'

baseFigureWidth = 700
baseFigureHeight = 350

### glob all the log files into a list, then create a dataframe with all json data

In [4]:
bwa_normal_logs = glob("../logs/openfaas/kops_multi-tenant_1x_8vcpu_16gb/bwa/*normal*")
normal_df_list = []
for log in bwa_normal_logs:
    with open(log) as f:
        obj = json.load(f)
    # cpuInfo is messing up the dataframe... remove for now
    obj.pop('cpuInfo')
    normal_df_list.append(pd.DataFrame([obj]))
normal_df = pd.concat(normal_df_list, ignore_index=True)

bwa_tumor_logs = glob("../logs/openfaas/kops_multi-tenant_1x_8vcpu_16gb/bwa/*tumor*")
tumor_df_list = []
for log in bwa_tumor_logs:
    with open(log) as f:
        obj = json.load(f)
    # cpuInfo is messing up the dataframe... remove for now
    obj.pop('cpuInfo')
    tumor_df_list.append(pd.DataFrame([obj]))
tumor_df = pd.concat(tumor_df_list, ignore_index=True)

In [5]:
normal_df.keys()

Index(['architecture', 'bootTime', 'cpuCores', 'cpuGuest', 'cpuGuestDelta',
       'cpuGuestNice', 'cpuGuestNiceDelta', 'cpuIOWait', 'cpuIOWaitDelta',
       'cpuIdle', 'cpuIdleDelta', 'cpuIrq', 'cpuIrqDelta', 'cpuKernel',
       'cpuKernelDelta', 'cpuModel', 'cpuNice', 'cpuNiceDelta', 'cpuSoftIrq',
       'cpuSoftIrqDelta', 'cpuSteal', 'cpuStealDelta', 'cpuType', 'cpuUser',
       'cpuUserDelta', 'endTime', 'frameworkRuntime', 'frameworkRuntimeDeltas',
       'freeMemory', 'functionName', 'host_name', 'http_foward', 'http_host',
       'http_start_time', 'initializationTime', 'invocations', 'lang',
       'linuxVersion', 'newcontainer', 'pageFaults', 'pageFaultsDelta',
       'platform', 'runtime', 'startTime', 'totalMemory', 'userRuntime',
       'uuid', 'version'],
      dtype='object')

# Make bar plots to profile bwa function data

In [11]:
fig = make_subplots(specs = [[{"secondary_y": False}]])

# Add figure title
fig.update_layout(
    barmode='stack',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.47
    ),
    margin=dict(
        t=0,
        b=1,
        l=1,
        r=1,
        autoexpand=True
    ),
    font=dict(
        size=16
    ),
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    plot_bgcolor=bgColor
)

profileData = pd.DataFrame()
profileData['payload'] = ["Normal", "Tumor"]
# Metrics for profiling
metrics = ['SoftIrq', 'Kernel', 'IOWait', 'User', 'Idle']
profileData['cpuTotalTime'] = [0, 0]
for metric in metrics:
    normalMean = normal_df[f'cpu{metric}Delta'].mean()
    tumorMean = tumor_df[f'cpu{metric}Delta'].mean()
    profileData[metric] = [normalMean, tumorMean]
    profileData['cpuTotalTime'] = [
        normalMean + profileData['cpuTotalTime'][0],
        tumorMean + profileData['cpuTotalTime'][1]
    ]

#profileData['runtime'] = [round(normal_df['runtime'].mean() / 1000), round(tumor_df['runtime'].mean() / 1000)]
#profileData['runtimeText'] = [f"Runtime:<br>{value} s" for value in profileData['runtime']]
for i, metric in enumerate(metrics):
    profileData[metric] = [
        profileData[metric][0] * 100 / profileData['cpuTotalTime'][0],
        profileData[metric][1] * 100 / profileData['cpuTotalTime'][1]
    ]
    text = [f'{round(val, 2)}%' for val in profileData[metric]]
    fig.add_trace(go.Bar(x = profileData["payload"],
                         y = profileData[metric],
                         name = f"CPU {metric}", marker_color=colorPallet[i],
                         text = text),
                  secondary_y=False)

# Set x-axis title
fig.update_xaxes(title_text="Payloads")

# Set y-axes titles
fig.update_yaxes(title_text="CPU Time Percentage", secondary_y=False)
# fig.update_yaxes(title_text="Function Runtime (ms)", secondary_y=True)

# py.iplot(fig)

fig.show()
# Save graph
# fig.write_image("graphs/bwa_profile_bars.pdf", width=baseFigureWidth, engine="kaleido", height=baseFigureHeight)